# Setup Monk

 - Run these commands
     
     - git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git
     
     - cd Monk_Object_Detection/5_pytorch_retinanet/installation
     
 - Select the right requirements file and run
 
     - cat requirements_cuda.txt | xargs -n 1 -L 1 pip install

In [0]:
!git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git

Cloning into 'Monk_Object_Detection'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 2245 (delta 60), reused 70 (delta 31), pack-reused 2118
Receiving objects: 100% (2245/2245), 97.06 MiB | 40.01 MiB/s, done.
Resolving deltas: 100% (283/283), done.
Checking out files: 100% (3781/3781), done.


In [0]:
cd Monk_Object_Detection/5_pytorch_retinanet/installation

/content/Monk_Object_Detection/5_pytorch_retinanet/installation/Monk_Object_Detection/5_pytorch_retinanet/installation


In [0]:
cat requirements.txt | xargs -n 1 -L 1 pip install

  Using cached https://files.pythonhosted.org/packages/ae/05/50a05de5337f7a924bb8bd70c6936230642233e424d6a9747ef1cfbde353/torch-1.3.0-cp36-cp36m-manylinux1_x86_64.whl
ERROR: torchvision 0.4.2 has requirement torch==1.3.1, but you'll have torch 1.3.0 which is incompatible.
  Found existing installation: torch 1.3.1
    Uninstalling torch-1.3.1:
      Successfully uninstalled torch-1.3.1
  Using cached https://files.pythonhosted.org/packages/88/95/90e8c4c31cfc67248bf944ba42029295b77159982f532c5689bcfe4e9108/torch-1.3.1-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: torch 1.3.0
    Uninstalling torch-1.3.0:
      Successfully uninstalled torch-1.3.0


# Download Dataset

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!unzip -uq "/content/drive/My Drive/PRImA Layout Analysis Dataset.zip" -d "/content/Monk_Object_Detection/5_pytorch_retinanet/installation"

# Prepare Dataset

In [0]:
!pip install tqdm -U

## VOC Format

## Dataset Directory Structure

    ../PRImA Layout Analysis Dataset/ (root_dir)
          |
          |-----------Images (img_dir)
          |              |
          |              |------------------img1.tif
          |              |------------------img2.tif
          |              |------------------.........(and so on)
          |
          |
          |-----------Annotations (anno_dir)
          |              |
          |              |------------------img1.xml
          |              |------------------img2.xml
          |              |------------------.........(and so on)

## Monk Format

## Dataset Directory Structure

    ../PRImA Layout Analysis Dataset (root)
          |
          |-----------Images (img_dir)
          |              |
          |              |------------------img1.jpg
          |              |------------------img2.jpg
          |              |------------------.........(and so on)
          |
          |
          |-----------train_labels.csv (anno_file)
          
          
## Annotation file format

           | Id         | Labels  | image_width | image_height                               |
           | img1.jpg   | x1 y1 x2 y2 label1 x1 y1 x2 y2 label2  | width | height
           
           
- Labels:  xmin ymin xmax ymax label
- xmin, ymin - top left corner of bounding box
- xmax, ymax - bottom right corner of bounding box

## COCO Format

## Dataset Directory Structure

    ../sample_dataset (root_dir)
          |
          |------PRImA Layout Analysis Dataset (coco_dir) 
          |         |
          |         |---Images (img_dir)
          |         |----|
          |              |-------------------img1.jpg
          |              |-------------------img2.jpg
          |              |-------------------.........(and so on)
          |
          |
          |         |---annotations (anno_dir)
          |         |----|
          |              |--------------------instances_Images.json 
          |              |--------------------classes.txt
          
          
 - instances_Train.json -> In proper COCO format
 - classes.txt          -> A list of classes in alphabetical order

## VOC to Monk type

In [0]:
import os
import sys
import numpy as np
import pandas as pd

import xmltodict
import json

from tqdm.notebook import tqdm

from pycocotools.coco import COCO

Requirement already up-to-date: tqdm in /usr/local/lib/python3.6/dist-packages (4.42.0)


In [0]:
Images_jpg_folder ="PRImA Layout Analysis Dataset/Images_jpg/"

In [0]:
if not os.path.isdir(Images_jpg_folder):
    os.mkdir(Images_jpg_folder);

In [0]:
from PIL import Image
img_folder = "PRImA Layout Analysis Dataset/Images"
Image_list = os.listdir(img_folder)
for name in Image_list:
    im = Image.open(img_folder+'/'+name)
    name = str(name).rstrip(".tif")
    im.save(Images_jpg_folder+name + '.jpg', 'JPEG')

In [0]:
root_dir = "PRImA Layout Analysis Dataset/";
img_dir = "Images/";
anno_dir = "XML/";
anno_folder = "PRImA Layout Analysis Dataset/XML"
img_folder = "PRImA Layout Analysis Dataset/Images_jpg"

In [0]:
combined = []
all_filenames = os.listdir(anno_folder)
for filename in all_filenames:
    annoFile = "{}/{}".format(anno_folder, filename)
    f = open(annoFile, 'r')
    my_xml = f.read()
    if len(filename) == 12:
      image_name = filename[0:8]+'.jpg'
    if len(filename) == 15:
      image_name = filename.split("-")[1]
      image_name = image_name[0:8]+'.jpg'
    label_str = ""
    object = ['SeparatorRegion','NoiseRegion','TextRegion','ImageRegion']
    for i in object:
        try:
            image_width= dict(dict(dict(xmltodict.parse(my_xml))['PcGts'])['Page'])['@imageWidth']
            image_height = dict(dict(dict(xmltodict.parse(my_xml))['PcGts'])['Page'])['@imageHeight']
            p = dict(dict(dict(xmltodict.parse(my_xml))['PcGts'])['Page'])[i]  
            try:
                a = dict(dict(dict(dict(dict(xmltodict.parse(my_xml))['PcGts'])['Page'])[i])['Coords'])['Point']
                x = []
                y = []
                for j in range(len(a)):
                    x.append(int(dict(a[j])['@x']))
                    y.append(int(dict(a[j])['@y']))
                x1 = str(min(x))
                y1 = str(min(y))
                x2 = str(max(x))
                y2 = str(max(y))
                label_str += x1 + " " + y1 + " " + x2 + " " + y2 + " " + i + " "

            except:

                b = dict(dict(dict(xmltodict.parse(my_xml))['PcGts'])['Page'])[i]
                for k in range(len(b)):
                    c = b[k]
                    d = dict(dict(c)['Coords'])['Point']
                    x = []
                    y = []
                    for l in range(len(a)):
                        x.append(int(dict(a[l])['@x']))
                        y.append(int(dict(a[l])['@y']))
                    x1 = str(min(x))
                    y1 = str(min(y))
                    x2 = str(max(x))
                    y2 = str(max(y))
                    label_str += x1 + " " + y1 + " " + x2 + " " + y2 + " " + i + " "
        except:
            continue
    label_str = label_str.strip()
    combined.append([image_name, label_str,image_width,image_height]) 

In [0]:
df = pd.DataFrame(combined, columns = ['ID', 'Label','image_width','image_height'])
df['Label'] = df['Label'].replace('',np.NaN)
df = df[pd.notnull(df['Label'])]
df = df.reset_index(drop=True)
df.to_csv(root_dir+"/train_labels.csv", index=False)

In [0]:
columns = df.columns

In [0]:
import os
Image_list = os.listdir(img_folder)
a = df[columns[0]].tolist()
b = [x for x in Image_list if x not in a]
for file in b:
  os.remove(img_folder + '/' + file) 

## Monk Type to COCO




In [0]:
import os
import numpy as np 
import cv2
import dicttoxml
import xml.etree.ElementTree as ET
from xml.dom.minidom import parseString
from tqdm import tqdm
import shutil
import json
import pandas as pd

In [0]:
root = "PRImA Layout Analysis Dataset/";
img_dir = "Images_jpg/";
anno_file = "train_labels.csv";

In [0]:
dataset_path = root;
images_folder = root + img_dir;
annotations_path = root + "annotations/";

In [0]:
if not os.path.isdir(annotations_path):
    os.mkdir(annotations_path)
    
input_images_folder = images_folder;
input_annotations_path = root + anno_file;

In [0]:
output_dataset_path = root;
output_image_folder = input_images_folder;
output_annotation_folder = annotations_path;

tmp = img_dir.replace("/", "");
output_annotation_file = output_annotation_folder + "/instances_" + tmp + ".json";
output_classes_file = output_annotation_folder + "/classes.txt";

In [0]:
if not os.path.isdir(output_annotation_folder):
    os.mkdir(output_annotation_folder);

In [0]:
df = pd.read_csv(input_annotations_path);
columns = df.columns

In [0]:
list_dict = [];
anno = [];
for i in range(len(df)):
    img_name = df[columns[0]][i];
    labels = df[columns[1]][i];
    tmp = labels.split(" ");
    for j in range(len(tmp)//5):
        label = tmp[j*5+4];
        if(label not in anno):
            anno.append(label);
    anno = sorted(anno)
    
for i in tqdm(range(len(anno))):
    tmp = {};
    tmp["supercategory"] = "master";
    tmp["id"] = i;
    tmp["name"] = anno[i];
    list_dict.append(tmp);

anno_f = open(output_classes_file, 'w');
for i in range(len(anno)):
    anno_f.write(anno[i] + "\n");
anno_f.close();

In [0]:
coco_data = {};
coco_data["type"] = "instances";
coco_data["images"] = [];
coco_data["annotations"] = [];
coco_data["categories"] = list_dict;
image_id = 0;
annotation_id = 0;


for i in tqdm(range(len(df))):
    img_name = df[columns[0]][i];
    labels = df[columns[1]][i];
    tmp = labels.split(" ");
    image_in_path = input_images_folder  + img_name;
    #print(image_in_path)


    images_tmp = {};
    images_tmp["file_name"] = img_name;
    images_tmp["height"] = int(df[columns[3]][i]);
    images_tmp["width"] = int(df[columns[2]][i]);
    images_tmp["id"] = image_id;
    coco_data["images"].append(images_tmp);
    

    for j in range(len(tmp)//5):
        x1 = int(tmp[j*5+0]);
        y1 = int(tmp[j*5+1]);
        x2 = int(tmp[j*5+2]);
        y2 = int(tmp[j*5+3]);
        label = tmp[j*5+4];
        print(x1,y1,x2,y2)
        annotations_tmp = {};
        annotations_tmp["id"] = annotation_id;
        annotation_id += 1;
        annotations_tmp["image_id"] = image_id;
        annotations_tmp["segmentation"] = [];
        annotations_tmp["ignore"] = 0;
        annotations_tmp["area"] = (x2-x1)*(y2-y1);

        annotations_tmp["iscrowd"] = 0;
        annotations_tmp["bbox"] = [x1, y1, x2-x1, y2-y1];
        annotations_tmp["category_id"] = anno.index(label);
        print( annotations_tmp["area"],annotations_tmp["bbox"])

        coco_data["annotations"].append(annotations_tmp)
    image_id += 1;

outfile =  open(output_annotation_file, 'w');
json_str = json.dumps(coco_data, indent=4);
outfile.write(json_str);
outfile.close();

# Training

In [0]:
import os
import sys
sys.path.append("../../5_pytorch_retinanet/lib/");

In [0]:
from train_detector import Detector

In [0]:
gtf = Detector();

In [0]:
root_dir = "./";
coco_dir = "PRImA Layout Analysis Dataset";
img_dir = "./";
set_dir = "Images_jpg";

In [0]:
gtf.Train_Dataset(root_dir, coco_dir, img_dir, set_dir, batch_size=2, use_gpu=True)

In [0]:
gtf.Model(model_name="resnet50");

In [0]:
gtf.Set_Hyperparams(lr=0.0001, val_interval=1, print_interval=20)

In [0]:
gtf.Train(num_epochs=1, output_model_name="final_model.pt");

# Inference

In [0]:
import os
import sys
sys.path.append("../../5_pytorch_retinanet/lib/");

In [0]:
from infer_detector import Infer

In [0]:
gtf = Infer();

In [0]:
gtf.Model(model_path="final_model.pt");

In [0]:
class_list = ['ImageRegion','NoiseRegion','TextRegion','SeparatorRegion']

In [0]:
img_folder = "sample_test";
gtf.predict_batch_of_images(img_folder, class_list, vis_threshold=0.001);
from IPython.display import Image
Image(filename='Inference/00000085.jpg')